# KCNH5 Individuals

In [1]:
import pandas as pd
from IPython.display import display, HTML
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.54


In [2]:
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
created_by="ORCID:0000-0002-0736-9199"
print(f"HPO version {hpo_version}")

HPO version 2024-02-27


In [3]:
df = pd.read_excel("input/KCNH5_individuals.xlsx")
df.head(2)

,PMID,title,individual_id,comment,disease_id,disease_label,transcript,allele_1,allele_2,variant.comment,...,Motor delay,Delayed speech and language development,Developmental regression,Intellectual disability,Hypotonia,Hyperintensity of cerebral white matter on MRI,Chiari type I malformation,Cerebral atrophy,Delayed CNS myelination,Miscellaneous
0,str,str,str,optional str,str,str,str,str,str,optional str,...,HP:0001270,HP:0000750,HP:0002376,HP:0001249,HP:0001252,HP:0030890,HP:0007099,HP:0002059,HP:0002188,Miscellaneous
1,PMID:23647072,Exome sequencing reveals new causal mutations in children with epileptic encephalopathies,Proband F,NaN,OMIM:620537,Developmental and epileptic encephalopathy 112,NM_139318.5,c.980G>A,na,R327H,...,observed,observed,P3Y,na,P3Y,na,na,na,na,Autistic behavior


In [4]:
encoder = CaseTemplateEncoder(df=df, hpo_cr=hpo_cr, created_by=created_by)
individuals = encoder.get_individuals()

Created encoders for 39 fields


In [5]:
KCNH5_transcript = "NM_139318.5"

vmanager = VariantManager(df=df,
                          individual_column_name="individual_id",
                          gene_symbol="KCNH5",
                          transcript=KCNH5_transcript,
                          allele_1_column_name="allele_1")

In [6]:
vmanager.to_summary()

,status,count,alleles
0,mapped,8,"c.1402A>C, c.2020-4A>G, c.970A>G, c.962G>A, c.1388T>C, c.1412T>C, c.998G>A, c.980G>A"
1,unmapped,0,


In [7]:
vmanager.add_variants_to_individuals(individuals)

In [8]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1, allelic_requirement=AllelicRequirement.MONO_ALLELIC)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Level,Error category,Count
ERROR,CONFLICT,1
WARNING,REDUNDANT,55


In [9]:
individuals = cvalidator.get_error_free_individual_list()
table = IndividualTable(individuals)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
Proband F (MALE; P13Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.980G>A (heterozygous),P6M: Bilateral tonic-clonic seizure (HP:0002069)P3Y: Developmental regression (HP:0002376); Hypotonia (HP:0001252)Focal hemiclonic seizure (HP:0006813); Epileptic encephalopathy (HP:0200134); Motor delay (HP:0001270); Delayed speech and language development (HP:0000750); Autistic behavior (HP:0000729); excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Infantile spasms (HP:0012469); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133)
patient 25 (MALE; P61Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.980G>A (heterozygous),Bilateral tonic-clonic seizure (HP:0002069); Epileptic encephalopathy (HP:0200134); Intellectual disability (HP:0001249); Hyperintensity of cerebral white matter on MRI (HP:0030890); Macroglossia (HP:0000158); excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Infantile spasms (HP:0012469); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133); excluded: Developmental regression (HP:0002376); excluded: Hypotonia (HP:0001252)
Case 1 (MALE; P4Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.980G>A (heterozygous),P8M: Bilateral tonic-clonic seizure (HP:0002069)Delayed speech and language development (HP:0000750); Autistic behavior (HP:0000729); excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Infantile spasms (HP:0012469); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133); excluded: Motor delay (HP:0001270); excluded: Hyperintensity of cerebral white matter on MRI (HP:0030890)
Case 2 (FEMALE; P4Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.2020-4A>G (heterozygous),Focal-onset seizure (HP:0007359); Infantile spasms (HP:0012469); Clonic seizure (HP:0020221); excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133); excluded: Hyperintensity of cerebral white matter on MRI (HP:0030890)
Case 3 (MALE; P2Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.962G>A (heterozygous),P7M: Clonic seizure (HP:0020221)excluded: Bilateral tonic-clonic seizure with focal onset (HP:0007334); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Infantile spasms (HP:0012469); excluded: Myoclonic seizure (HP:0032794); excluded: Generalized non-motor (absence) seizure (HP:0002121); excluded: Status epilepticus (HP:0002133); excluded: Motor delay (HP:0001270); excluded: Delayed speech and language development (HP:0000750); excluded: Developmental regression (HP:0002376); excluded: Intellectual disability (HP:0001249); excluded: Hypotonia (HP:0001252); excluded: Hyperintensity of cerebral white matter on MRI (HP:0030890)
Proband 1 (MALE; P4Y),Developmental and epileptic encephalopathy 112 (OMIM:620537),NM_139318.5:c.970A>G (heterozygous),P3M: Focal motor seizure (HP:0011153)Bilateral tonic-clonic seizure with focal onset (HP:0007334); Status epilepticus (HP:0002133); Epileptic encephalopathy (HP:0200134); Global developmental delay (HP:0001263); excluded: Focal impaired awareness motor seizure (HP:0032712); excluded: Focal aware motor seizure (HP:0020217); excluded: Focal tonic seizure (HP:0011167); excluded: Focal hemiclonic seizure (HP:0006813); excluded: Myoclonic seiz

In [10]:
encoder.output_individuals_as_phenopackets(individual_list=individuals)

We output 22 GA4GH phenopackets to the directory phenopackets


# HPOA file

In [11]:
ingestor = PhenopacketIngestor(indir="phenopackets")
ppkt_d = ingestor.get_phenopacket_dictionary()
ppkt_list = list(ppkt_d.values())

[pyphetools] Ingested 22 GA4GH phenopackets.


In [12]:
builder = HpoaTableBuilder(phenopacket_list=ppkt_list)

In [13]:
PMID = "PMID:36307226"
hpoa_table_creator = builder.autosomal_dominant(PMID).build()
hpoa_table_creator.get_dataframe()

We found a total of 27 unique HPO terms
Extracted disease: Developmental and epileptic encephalopathy 112 (OMIM:620537)


,#diseaseID,diseaseName,phenotypeID,phenotypeName,onsetID,onsetName,frequency,sex,negation,modifier,description,publication,evidence,biocuration
0,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0002069,Bilateral tonic-clonic seizure,,,10/10,,,,,PMID:36307226,PCS,ORCID:0000-0002-0736-9199
1,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0000750,Delayed speech and language development,,,9/9,,,,,PMID:36307226,PCS,ORCID:0000-0002-0736-9199
2,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0007334,Bilateral tonic-clonic seizure with focal onset,,,6/16,,,,,PMID:36307226,PCS,ORCID:0000-0002-0736-9199
3,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0032712,Focal impaired awareness motor seizure,,,4/16,,,,,PMID:36307226,PCS,ORCID:0000-0002-0736-9199
4,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0020217,Focal aware motor seizure,,,1/17,,,,,PMID:36307226,PCS,ORCID:0000-0002-0736-9199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0003584,Late onset,,,1/17,,,,,PMID:36307226,PCS,ORCID:0000-0002-0736-9199
68,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0003623,Neonatal onset,,,1/1,,,,,PMID:32725632,PCS,ORCID:0000-0002-0736-9199
69,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0003593,Infantile onset,,,1/1,,,,,PMID:23647072,PCS,ORCID:0000-0002-0736-9199
70,OMIM:620537,Developmental and epileptic encephalopathy 112,HP:0003593,Infantile onset,,,3/3,,,,,PMID:35874597,PCS,ORCID:0000-0002-0736-9199


In [14]:
hpoa_table_creator.write_data_frame()

Wrote HPOA disease file to OMIM-620537.tab
